# Train S4 on Colab
Please skip the [WIP] section. This notebook can only run with the current version of Colab, i.e. [2025-01-13](https://colab.research.google.com/notebooks/relnotes.ipynb#scrollTo=414ec961):
- `python` 3.11
- `torch` 2.5.1
- `cuda` 12.4

The cell outputs in this section is for reference.

In [2]:
!uname -m && cat /etc/*release

x86_64
DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=22.04
DISTRIB_CODENAME=jammy
DISTRIB_DESCRIPTION="Ubuntu 22.04.4 LTS"
PRETTY_NAME="Ubuntu 22.04.4 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.4 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [3]:
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [4]:
!nvidia-smi

Fri Feb  7 10:42:34 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
!pip list | grep torch

torch                              2.5.1+cu124
torchaudio                         2.5.1+cu124
torchsummary                       1.5.1
torchvision                        0.20.1+cu124


In [5]:
!pip check

ipython 7.34.0 requires jedi, which is not installed.
pygobject 3.42.1 requires pycairo, which is not installed.
torch 2.5.1+cu124 has requirement nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2.
torch 2.5.1+cu124 has requirement nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82.
torch 2.5.1+cu124 has requirement nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82.
torch 2.5.1+cu124 has requirement nvidia-cuda-runtime-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-runtime-cu12 12.5.82.
torch 2.5.1+cu124 has requirement nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75.
torch 2.5.1+cu124 has require

The inconsistent `nvidia` packages will be fixed by `pip install pytorch-lightning` soon.

## [WIP] Upgrade CUDA (SKIP ME!)

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.1-1_all.deb

In [ ]:
!sudo dpkg -i cuda-keyring_1.1-1_all.deb

In [ ]:
!sudo apt-get update

In [ ]:
!sudo apt-get -y install cuda-toolkit-12-6

In [ ]:
!sudo apt-get install -y nvidia-open

In [ ]:
!sudo update-alternatives --display cuda

In [ ]:
!sudo update-alternatives --config cuda

In [ ]:
!ls -alh /usr/local/cuda-12.6/targets/x86_64-linux/lib

In [ ]:
!echo $PATH

In [ ]:
%env PATH=/usr/local/cuda-12.6/bin:/usr/bin
!echo $PATH

In [ ]:
!echo $LD_LIBRARY_PATH

In [ ]:
%env LD_LIBRARY_PATH=/usr/local/cuda-12.6/lib64
!echo $LD_LIBRARY_PATH

In [ ]:
!lsmod | grep nvidia

In [ ]:
!sudo rmmod nvidia_drm
!sudo rmmod nvidia_modeset
!sudo rmmod nvidia_uvm

In [ ]:
!sudo rmmod nvidia

In [ ]:
!sudo lsof /dev/nvidia*

In [ ]:
!pip uninstall -y fastai

In [ ]:
!pip uninstall -y torch torchaudio torchsummary torchvision

In [ ]:
!pip install torch torchaudio torchvision --index-url https://download.pytorch.org/whl/cu126

In [ ]:
!pip list | grep nvidia

## Install minimum deps
Many deps are already built-in. This is just a Colab shortcut. We will have an updated `requirements.txt` in the near future.

In [ ]:
!pip install pytorch-lightning

In [ ]:
!pip check

In [ ]:
!pip install datasets

In [ ]:
!pip install "gcsfs<2024.10.0"

In [ ]:
!pip check

***Better restart the Colab session after installing `hydra-core` below.***

In [ ]:
!pip install hydra-core

The deps below seem unnecessary for `mnist` and yet I don't know whether they are required for `imdb`.

In [ ]:
!pip install torchdata

In [2]:
!pip install spacy
!python -m spacy download en_core_web_sm

In [ ]:
!pip install sacremoses

## Compile `torchtext`

In [ ]:
!pip install ninja

In [ ]:
%cd /content

In [6]:
!rm -rf torchtext

In [ ]:
!git clone https://github.com/pytorch/text torchtext

In [ ]:
%cd /content/torchtext

In [ ]:
!git submodule update --init --recursive

Compiling `torchtext` will take about 4.5 minutes.

**Caveat**: This compilation may change some CUDA deps.

In [ ]:
!python setup.py clean install

In [ ]:
!pip list | grep torch

In [ ]:
!pip check

## Train S4 with IMDB

In [14]:
%cd /content

/content


In [15]:
!rm -rf s4

The hard-coded repo and branch below should be changed in the future.

In [ ]:
!git clone -b build/colab_poetry https://github.com/tianjianjiang/s4.git

In [ ]:
%cd /content/s4

Remember to put your own W&B API key to Colab's Secrets.

In [18]:
from google.colab import userdata


wandb_api_key = userdata.get("WANDB_API_KEY")

In [ ]:
%env WANDB_API_KEY={wandb_api_key}

In [20]:
!python -m train pipeline=imdb model=s4

CONFIG
├── train
│   └── seed: 0                                                                                     
│       name: null                                                                                  
│       interval: epoch                                                                             
│       monitor: val/accuracy                                                                       
│       mode: max                                                                                   
│       ema: 0.0                                                                                    
│       test: false                                                                                 
│       debug: false                                                                                
│       ignore_warnings: false                                                                      
│       state:                                                            